<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-상장법인-목록-읽기" data-toc-modified-id="1.-상장법인-목록-읽기-1">1. 상장법인 목록 읽기</a></span></li><li><span><a href="#2.-종목코드별-PER,-PBR-스크래핑" data-toc-modified-id="2.-종목코드별-PER,-PBR-스크래핑-2">2. 종목코드별 PER, PBR 스크래핑</a></span></li></ul></div>

# 1. 상장법인 목록 읽기

- **전체 종목코드 데이터 수집** : 한국거래소 기업공시채널(kind.krx.co.kr) - 상장법인상세정보 - 상장법인목록 - EXCEL 파일 다운로드

In [1]:
import pandas as pd

In [2]:
krx_list = pd.read_html('./data/상장법인목록.xls')
krx_list[0].tail()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
2406,CJ대한통운,120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,박근희,http://www.cjlogistics.com,서울특별시
2407,메리츠화재,60,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
2408,경방,50,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
2409,유수홀딩스,700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시
2410,한진중공업홀딩스,3480,연료용 가스 제조 및 배관공급업,지주회사,1956-03-03,12월,"조남호, 조원국",http://www.hhic-holdings.com,경기도


종목코드의 앞 자리 0이 모두 사라져 여섯 자리가 아닌 행들이 있으므로, map() 함수를 사용해 이를 보정

In [3]:
krx_list[0].종목코드 = krx_list[0].종목코드.map('{:06d}'.format)
krx_list[0].tail()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
2406,CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,박근희,http://www.cjlogistics.com,서울특별시
2407,메리츠화재,000060,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
2408,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
2409,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시
2410,한진중공업홀딩스,003480,연료용 가스 제조 및 배관공급업,지주회사,1956-03-03,12월,"조남호, 조원국",http://www.hhic-holdings.com,경기도


# 2. 종목코드별 PER, PBR 스크래핑

- **Company Guide**(https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A종목코드&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701) 사이트에서 **PER, PBR 데이터 수집**

In [4]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import re

In [11]:
# for code in tqdm(krx_list[0].종목코드):
#     cg_url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701'

#     html = BeautifulSoup(requests.get(cg_url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
#     stxt1 = re.sub('\xa0', ' ', html.find("span", class_="stxt stxt1").text)
#     stxt2 = re.sub('\xa0', ' ', html.find("span", class_="stxt stxt2").text)
#     corp_group = html.find("div", class_="corp_group2")
#     PER = corp_group.find_all('dd')[1].text
#     업종PER = corp_group.find_all('dd')[5].text
#     PBR = corp_group.find_all('dd')[7].text

-  결측치가 많은 관계로 **네이버 금융(https://finance.naver.com/item/main.nhn?code=종목코드) 에서 추가로 수집**

In [10]:
data = []

for code in tqdm(krx_list[0].종목코드):
    n_url = f'https://finance.naver.com/item/main.nhn?code={code}'
    html = BeautifulSoup(requests.get(n_url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
    
    # 업종명 수집
    try:
        group = html.find('h4', class_='h_sub sub_tit7')
        업종 = group.find('a').text
    except:
        업종 = 'N/A'
        
    # PER, PBR 수집
    try:
        per = html.find('em', id='_per').text
        pbr = html.find('em', id='_pbr').text
    except:
        per, pbr = 'N/A', 'N/A'
        
    # 동일업종 PER 수집
    try:
        table = html.find('table', summary='동일업종 PER 정보')
        업종per = table.find('em').text
    except:
        업종per = 'N/A'
    
    # data 리스트에 데이터 추가
    data.append([업종, 업종per, per, pbr])
    
df = pd.DataFrame(data)
df.columns = ['업종2', '업종PER', 'PER', 'PBR']

100%|██████████████████████████████████████████████████████████████████████████████| 2411/2411 [15:10<00:00,  2.65it/s]


In [15]:
krx_df = krx_list[0][['회사명', '종목코드', '업종']]

result = pd.concat([krx_df, df], axis=1)
result.tail()

,회사명,종목코드,업종,업종2,업종PER,PER,PBR
2406,CJ대한통운,000120,도로 화물 운송업,항공화물운송과물류,12.60,31.28,1.02
2407,메리츠화재,000060,보험업,손해보험,6.87,4.41,0.61
2408,경방,000050,종합 소매업,"섬유,의류,신발,호화품",33.35,7.49,0.43
2409,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,항공화물운송과물류,12.60,18.92,0.62
2410,한진중공업홀딩스,003480,연료용 가스 제조 및 배관공급업,가스유틸리티,19.09,52.24,0.31


In [16]:
result.to_csv('./data/네이버금융_PER,PBR.csv', index=False)